In [12]:
import spacy
import nltk
from pprint import pprint
import random
from sklearn import preprocessing
import numpy as np
nltk.download("all", quiet=True)

nlp = spacy.load("en_core_web_sm")


In [13]:
import os
def load_corpus(folder):
    corpus = []
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in files:
            try:
                with open(os.path.join(root, name), "r") as fp:
                    corpus.append(fp.read())
            except UnicodeDecodeError as e:
                print(e.__str__(), "for", os.path.join(root, name))
    return corpus

def load_corpuses(folder):
    sub_folders = []
    for root, dirs, files in os.walk(folder):
        if dirs:
            for dir_ in dirs:
                sub_folders.append(dir_)

    corpuses = {}
    print(sub_folders)
    for sub_folder in sub_folders:
        print(os.path.join(folder, sub_folder))
        corpuses[sub_folder] = load_corpus(os.path.join(folder, sub_folder))
    return corpuses

corpuses = load_corpuses("bbc")

['business', 'tech', 'entertainment', 'sport', 'politics']
bbc/business
bbc/tech
bbc/entertainment
bbc/sport
'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte for bbc/sport/199.txt
bbc/politics


In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords

vectorizer = CountVectorizer(stop_words=stopwords.words('english'))
tfid = TfidfTransformer()

all_ = []
for corpus in corpuses:
    all_ += corpuses[corpus]

vectorizer.fit(all_)
tfid.fit(vectorizer.transform(all_))

# x = vectorizer.transform(corpuses["tech"])
# vectorizer.vocabulary_.get('Sony')

TfidfTransformer()

In [15]:
import math
x = []
y =[]
from tqdm.notebook import tqdm

entity_types = CountVectorizer(stop_words=stopwords.words('english'))
entity_types.fit(['CARDINAL', 'PERSON', 'GPE', 'MONEY', 'ORG', 'ORDINAL', 'WORK_OF_ART', 'NORP', 'PERCENT', 'DATE', 'LANGUAGE', 'FAC', 'LOC', 'TIME', 'PRODUCT', 'EVENT', 'QUANTITY', 'LAW'])
for corpus in tqdm(corpuses):
    for story in tqdm(corpuses[corpus]):
        analysed = nlp(story)
        # print(entity_types.transform([tag.label_ for tag in analysed.ents]).toarray()[0])
        # print(vectorizer.transform([story]).toarray()[0])
        # print(list(tfid.transform(vectorizer.transform([story])).toarray()[0]))
        x.append(list(vectorizer.transform([story]).toarray()[0]) + 
                 list(entity_types.transform([tag.label_ for tag in analysed.ents]).toarray()[0]) +
                 list(tfid.transform(vectorizer.transform([story])).toarray()[0])
                )
        # x.append(tfid.transform(vectorizer.transform([story])).toarray()[0])

        y.append(corpus)

import random

c = list(zip(x, y))

random.shuffle(c)

x, y = zip(*c)
# pprint(x[:3])
# pprint(y[:3])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/510 [00:00<?, ?it/s]

  0%|          | 0/401 [00:00<?, ?it/s]

  0%|          | 0/386 [00:00<?, ?it/s]

  0%|          | 0/510 [00:00<?, ?it/s]

  0%|          | 0/417 [00:00<?, ?it/s]

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

fs_sentanalysis=SelectKBest(chi2, k=500).fit(x, y)
x_chi = fs_sentanalysis.transform(x)

In [11]:
print(len(x), len(x_chi))

NameError: name 'x_chi' is not defined

In [5]:
size_dataset_full=len(x_chi)
size_test=int(round(size_dataset_full*0.2,0))

list_test_indices=random.sample(range(size_dataset_full), size_test)

test_x = []
test_y = []
train_x = []
train_y = []

for i,example in enumerate(x_chi):
  if i in list_test_indices:
      test_x.append(example)
      test_y.append(y[i])
  else:
      train_x.append(example)
      train_y.append(y[i])

le = preprocessing.LabelEncoder()
le.fit(train_y + test_y)
train_y = le.transform(train_y)
test_y = le.transform(test_y)

In [7]:
print(test_y)

[1 2 4 2 3 2 1 0 0 4 0 1 0 3 2 0 2 4 0 3 4 3 3 3 3 0 4 0 4 0 2 2 2 3 1 0 1
 0 3 4 0 0 2 0 1 3 4 0 0 3 3 0 0 3 2 4 0 1 0 0 4 1 3 0 1 4 0 4 4 1 1 4 2 3
 4 0 4 1 0 0 1 3 2 2 2 0 2 1 4 4 4 0 4 2 4 1 4 0 3 2 3 3 4 0 0 3 0 3 4 3 0
 2 2 0 2 3 0 3 2 2 3 0 4 1 3 2 0 0 3 1 3 3 1 4 1 4 4 3 0 0 4 4 2 4 3 4 2 3
 1 0 1 1 0 3 2 3 2 3 3 0 0 1 4 1 0 3 0 0 1 1 4 1 3 1 3 3 3 0 3 0 3 2 0 0 4
 0 3 1 1 4 4 4 4 4 2 4 3 2 2 1 0 3 0 3 4 0 3 1 0 3 0 2 0 1 2 3 4 0 1 3 1 0
 1 3 1 3 1 2 1 0 1 0 3 4 3 3 2 0 2 1 1 0 0 2 3 3 1 0 2 3 4 3 3 0 4 4 3 2 3
 1 2 4 0 3 3 4 4 4 2 0 0 4 0 0 0 0 1 3 3 2 3 0 2 0 2 0 0 0 4 3 3 4 1 1 4 4
 2 2 3 0 0 3 0 1 1 2 2 0 1 0 2 2 0 3 0 1 3 4 4 2 1 3 3 2 3 3 0 2 4 3 0 2 0
 1 0 4 1 0 0 0 1 3 0 2 0 0 0 2 1 1 0 1 3 1 0 2 3 1 3 4 2 1 3 3 4 3 0 4 4 2
 4 3 4 0 0 3 4 1 1 1 1 0 0 2 4 3 0 2 3 1 3 0 4 1 4 4 4 0 3 2 2 3 3 3 4 4 4
 0 1 0 0 1 2 4 0 1 3 2 2 2 3 0 0 0 0 0 2 3 2 0 4 0 4 1 3 2 2 1 3 2 1 2 0 0
 3]


In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

svm_clf=make_pipeline(StandardScaler(), svm.SVC(cache_size=10000, decision_function_shape='ovo'))

In [9]:
svm_clf.fit(train_x_chi, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(cache_size=10000, decision_function_shape='ovo'))])

In [10]:
Y_text_predictions = svm_clf.predict(test_x)

ValueError: X has 58572 features, but StandardScaler is expecting 500 features as input.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, Y_text_predictions))

In [ ]:
story = """
England centre Emily Scarratt says having a final in this year's reformatted Women's Six Nations could help the side's World Cup preparations.

Because of the coronavirus pandemic, teams will have two pool games and a final rather than playing all of the other five teams.

Covid-19 also forced the postponement of the World Cup until 2022.

"You do not get a final very often in our calendar," Scarratt told BBC Radio 5 Live's Rugby Union Weekly.

Listen to Rugby Union Weekly: Danny's coat, Scaz and Liar's dice
Shaunagh Brown column: Wasted chances, fantasy rugby and the importance of good hair
The 31-year-old - who captained England to a 52-10 win against Scotland on Saturday in the absence of Sarah Hunter - added that she "was not sure" about the new format initially.

"I am a traditionalist. I like the format of playing five games," she continued.

"But we now get the opportunity to play a final. Sometimes when you get to that moment it is a World Cup and all of a sudden a big deal."

England's opening victory puts them top of Pool A with a game in Italy to come on Saturday.

After France's dominant 53-0 defeat of Wales in Pool B, all signs currently point to a final between England and Les Bleues, to be shown live on BBC Two on 24 April.

Aside from the benefits of a final, Scarratt is glad that the Women's Six Nations is taking place in a different window to the men's tournament.

"We're not having to compete with men's kick-off times or a load of people watching the [men's] game at Twickenham and a much lower percentage watching the [women's] game at the Stoop," she explained.

"That is the exciting bit for me. It would be great to have crowds to quantify what that would look like but at the same time it is really important for us to stand on our own two feet this time around."
"""

analysed = nlp(story)
genre = le.inverse_transform([svm_clf.predict([list(vectorizer.transform([story]).toarray()[0]) + 
                                               list(entity_types.transform([tag.label_ for tag in analysed.ents]).toarray()[0]) +
                                               list(tfid.transform(vectorizer.transform([story])).toarray()[0])
                             ])[0]])[0]

display(genre)